# Capstone project - week 3: Segmenting and Clustering Toronto

In [178]:
# importing the required libraries
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# **1** Fetching data from wikipedia page <a href='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'>wikipedia page link</a>

In [134]:
# using beautiful soup 
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')

In [135]:
# finding table tag in soup object
table=soup.find('table',class_='wikitable sortable')

In [136]:
# initialzing the dataframe with three columns
neighbourhood=pd.DataFrame(columns=['Postal Code','Borough','Neighbourhood'])
neighbourhood

,Postal Code,Borough,Neighbourhood


# 2 Appending data to dataframe from table


In [138]:
i=-1
for a in table.find_all('tr'):
    list1=[]
    for b in a.find_all('td'):
        if('\n' in b.text):
            string=(b.text).split('\n')[0]
            list1.append(string)
        else:
            list1.append(b.text)
    if(i==-1):
        i=i+1
        continue # skip the first row with heading
    neighbourhood.loc[i]=list1  # adding row in each iteration
    i=i+1
neighbourhood.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# 3 Cleaning the dataframe
### removing rows with borough column as Not assigned.

In [139]:
neighbourhood=neighbourhood[neighbourhood['Borough'] != 'Not assigned']
neighbourhood.reset_index(drop=True,inplace=True)
neighbourhood.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### **combining Neighbourhood with same postcode**

In [140]:
neighbourhood['Neighbourhood']=neighbourhood.groupby('Postal Code')['Neighbourhood'].transform(lambda x :
', '.join(x)).values
neighbourhood.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"


In [141]:
# removing the duplicates that arises.
neighbourhood = neighbourhood.drop_duplicates().reset_index(drop=True)
neighbourhood.head(10)        

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


### **Replace Neighbourhoods with 'Not assigned' to their Burough name**

In [142]:
neighbourhood['Neighbourhood'].replace('Not assigned',neighbourhood['Borough'],inplace=True)
neighbourhood.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [143]:
# Shape of the dataframe
neighbourhood.shape

(103, 3)

# 4 Obtaining geospatial data
By importing the CSV file from <a href='https://cocl.us/Geospatial_data'>https://cocl.us/Geospatial_data</a>

In [144]:
lat_long_df=pd.read_csv('Geospatial_Coordinates.csv')
lat_long_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [145]:
lat_long_df.shape

(103, 3)

### Merging the two dataframe with respect to their **Postal Code**

In [146]:
neighbourhood=pd.merge(neighbourhood,lat_long_df, on='Postal Code')
neighbourhood.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [147]:
neighbourhood.shape

(103, 5)

# 5 Segmenting and Clustering Neighborhoods in Toronto

## filtering Neighbourhood only for city **Toronto**
filtering Neighbourhood that contains word **Toronto**

In [148]:
toronto_neighbourhood=neighbourhood[neighbourhood['Borough'].str.contains('Toronto')] 

In [149]:
toronto_neighbourhood.reset_index(drop=True,inplace=True)
print(toronto_neighbourhood.shape)
toronto_neighbourhood.head()

(38, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [150]:
# importing required libraries
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

### Using geopy to get latitude and longitude on city Toronto

In [151]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [152]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map
for lat, lng, borough,neighborhood,pc in zip(toronto_neighbourhood['Latitude'], toronto_neighbourhood['Longitude'],
                                             toronto_neighbourhood['Borough'], toronto_neighbourhood['Neighbourhood'],toronto_neighbourhood['Postal Code']):
    label = '{}, {}, {}'.format(neighborhood, borough,pc)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

## Define Foursquare Credentials and Version

In [154]:
CLIENT_ID = 'JLJ3QFTM1TLSW0SVAGKEIH5YMOMA5PAHEYCBVHFQARTAMAA4' # your Foursquare ID
CLIENT_SECRET = '1H2K25IZ40Q52PS53GIAYNM5SJLR4QLDBNC3HXFGDBNAC140' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET) 

Your credentails:
CLIENT_ID: JLJ3QFTM1TLSW0SVAGKEIH5YMOMA5PAHEYCBVHFQARTAMAA4
CLIENT_SECRET:1H2K25IZ40Q52PS53GIAYNM5SJLR4QLDBNC3HXFGDBNAC140


### Let's explore neighborhoods in Toronto.


we will use foursquare api to explore venues in each neighbourhood . below function gives nearby venues in neighbourhood.

In [156]:
def getNearbyVenues(post_codes,names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for post_code, name, lat, lng in zip(post_codes,names, latitudes, longitudes):
        print(post_code)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            post_code,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Post Code',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### code to run the above function on each neighborhood and create a new dataframe called Toronto_venues

In [157]:
LIMIT=100
Toronto_venues = getNearbyVenues(toronto_neighbourhood['Postal Code'],toronto_neighbourhood['Neighbourhood'],
                                 toronto_neighbourhood['Latitude'],toronto_neighbourhood['Longitude'])

M5A
M5B
M5C
M4E
M5E
M5G
M6G
M5H
M6H
M5J
M6J
M4K
M5K
M6K
M4L
M5L
M4M
M4N
M5N
M4P
M5P
M6P
M4R
M5R
M6R
M4S
M5S
M6S
M4T
M5T
M4V
M5V
M4W
M5W
M4X
M5X
M4Y
M7Y


In [158]:
Toronto_venues.shape

(1707, 8)

In [159]:
Toronto_venues.head()

,Post Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,M5A,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,M5A,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [160]:
# checking hoe many venues returned by each neighbourhood
Toronto_venues.groupby('Post Code').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Post Code,,,,,,,
M4E,5,5,5,5,5,5,5
M4K,42,42,42,42,42,42,42
M4L,20,20,20,20,20,20,20
M4M,39,39,39,39,39,39,39
M4N,4,4,4,4,4,4,4
M4P,10,10,10,10,10,10,10
M4R,21,21,21,21,21,21,21
M4S,36,36,36,36,36,36,36
M4T,4,4,4,4,4,4,4


In [161]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


In [162]:
# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Postcode column back to dataframe
toronto_onehot['Post Code'] = Toronto_venues['Post Code'] 

# move PostCode column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Post Code,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [163]:
toronto_onehot.shape

(1707, 239)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [164]:
toronto_grouped = toronto_onehot.groupby('Post Code').mean().reset_index()
toronto_grouped.head()

,Post Code,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,M4K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023810,...,0.0,0.02381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023810
2,M4L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,M4M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.051282,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025641
4,M4N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [165]:
# function to return required number of most common venues.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]  # Leaving the Post Code value in toronto_grouped
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [166]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Post Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Post Code'] = toronto_grouped['Post Code']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Post Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Coffee Shop,Pub,Neighborhood,Gym / Fitness Center,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Yoga Studio
1,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Brewery,Bakery,Fruit & Vegetable Store,Juice Bar,Spa
2,M4L,Park,Gym,Intersection,Pizza Place,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery
3,M4M,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Chinese Restaurant,Latin American Restaurant,Bookstore,Brewery
4,M4N,Park,Dim Sum Restaurant,Swim School,Bus Line,Yoga Studio,Donut Shop,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market


# 4. Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [175]:
k_cluster=5

toronto_grouped_cluster=toronto_grouped.drop('Post Code',axis=1)

cluster_model=KMeans(n_clusters=k_cluster)

cluster_model.fit(toronto_grouped_cluster)

print(cluster_model.labels_)

[0 2 2 2 2 2 2 2 1 0 1 2 2 2 2 2 2 2 2 2 2 2 3 4 2 2 2 2 2 2 2 2 2 2 2 2 2
 2]


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [176]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', cluster_model.labels_)

toronto_merged = toronto_neighbourhood

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Post Code'), on='Postal Code')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,2,Coffee Shop,Bakery,Park,Pub,Café,Restaurant,Theater,Mexican Restaurant,Breakfast Spot,Event Space
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bar,Theater,Ramen Restaurant,Plaza,Tea Room
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Restaurant,Café,Hotel,Clothing Store,Cosmetics Shop,Bakery,Italian Restaurant,Park,Cocktail Bar
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pub,Neighborhood,Gym / Fitness Center,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Yoga Studio
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Restaurant,Cocktail Bar,Café,Farmers Market,Pub,Seafood Restaurant,Cheese Shop,Beer Bar,Italian Restaurant


In [177]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_cluster)
ys = [i + x + (i*x)**2 for i in range(k_cluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, pc, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(pc) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters